## Librerias

In [33]:
import pandas as pd
import re
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
cantidad = 0
with open('../Datos/Originales/apache-logs.txt', 'r') as archivo:
    # Lee cada línea del archivo
    for linea in archivo:
        # Imprime cada línea
        cantidad +=1
print(cantidad)

13853


## Lectura

In [3]:
pattern = r'^(\S+) (\S+) (\S+) \[(.*?)\] "(\S+) (\S+) (\S+)" (\d+) (\d+) "([^"]+)" "([^"]+)"'

# Lista para almacenar los resultados
data = []

# Leer el archivo de texto línea por línea
with open('../Datos/Originales/apache-logs.txt', 'r') as file:
    for log_line in file:
        matches = re.match(pattern, log_line)
        if matches:
            client_ip, ident, auth, timestamp, method, request, hhtp_version, status_code, bytes_sent, referrer, user_agent = matches.groups()
            data.append([client_ip, ident, auth, timestamp, method, request, hhtp_version, status_code, bytes_sent, referrer, user_agent])
        else:
            print("No match found for line:", log_line)

# Crear DataFrame
df = pd.DataFrame(data, columns=['Client IP', 'Ident', 'Auth', 'Timestamp', 'Method', 'Request', 'HTTP Version', 'Status Code', 'Bytes Sent', 'Referrer', 'User Agent'])

# Mostrar el DataFrame
print(df)

No match found for line: "192.168.4.164 - - [22/Dec/2023:15:19:14 +0300] "GET /DVWA/ HTTP/1.1" 200 1964 "() { :;}; echo \"NS:\" $(/bin/sh -c \"expr 268409241 - 2\")" "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.16 Safari/537.36""

No match found for line: "192.168.4.164 - - [22/Dec/2023:15:19:15 +0300] "GET /DVWA/ HTTP/1.1" 200 1964 "'\"@--></style></scRipt><scRipt>netsparker(0x000008)</scRipt>" "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.16 Safari/537.36""

No match found for line: "192.168.4.164 - - [22/Dec/2023:15:19:15 +0300] "GET /DVWA/ HTTP/1.1" 200 1964 "'\"@--></style></scRipt><scRipt>netsparker(0x000009)</scRipt>" "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.16 Safari/537.36""

No match found for line: "192.168.4.164 - - [22/Dec/2023:15:19:15 +0300] "GET /DVWA/ HTTP/1.1" 200 1964 "'\"--></style></scRipt><scRipt>netsparker(0x000006)</s

In [7]:
df.head()

,Client IP,Ident,Auth,Timestamp,Method,Request,HTTP Version,Status Code,Bytes Sent,Referrer,User Agent
0,"""192.168.4.164",-,-,22/Dec/2023:15:19:05 +0300,GET,/DVWA/,HTTP/1.1,200,2020,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
1,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/,HTTP/1.1,200,263,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
2,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/.git/config,HTTP/1.1,404,139,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
3,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/.git/config,HTTP/1.1,404,139,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
4,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/.svn/all-wcprops,HTTP/1.1,404,139,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...


## Duplicidades

In [5]:
df[df.duplicated()]

,Client IP,Ident,Auth,Timestamp,Method,Request,HTTP Version,Status Code,Bytes Sent,Referrer,User Agent
3,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/.git/config,HTTP/1.1,404,139,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
5,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/.svn/all-wcprops,HTTP/1.1,404,139,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
7,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/.svn/wc.db,HTTP/1.1,404,139,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
9,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/~.aspx,HTTP/1.1,404,139,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
11,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/cgi-bin/,HTTP/1.1,404,139,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
...,...,...,...,...,...,...,...,...,...,...,...
13591,"""192.168.4.25",-,-,22/Dec/2022:16:38:55 +0300,POST,/index.php/component/users/?task=registration....,HTTP/1.1,200,237,http://192.168.4.161/DVWA,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
13595,"""192.168.4.25",-,-,22/Dec/2022:16:27:03 +0300,POST,/index.php/component/search/,HTTP/1.1,200,3053,-,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
13613,"""192.168.4.25",-,-,22/Dec/2022:16:29:16 +0300,POST,/administrator/index.php,HTTP/1.1,500,1522,-,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
13617,"""192.168.4.25",-,-,22/Dec/2022:16:22:11 +0300,POST,/index.php/component/search/,HTTP/1.1,303,376,http://192.168.4.161/DVWA,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...


In [6]:
Eliminar duplicidades???????????

SyntaxError: invalid syntax (1909389982.py, line 1)

Distribucion method

In [8]:
df['Method'].unique()

array(['GET', 'HEAD', 'NETSPARKER', 'POST', 'OPTIONS', 'PROPFIND'],
      dtype=object)

In [9]:
metodos = df['Method'].value_counts().reset_index()
metodos

,Method,count
0,POST,6689
1,GET,4826
2,HEAD,2038
3,OPTIONS,34
4,PROPFIND,34
5,NETSPARKER,1


In [18]:
fig = px.bar(metodos, x='Method', y='count', color='Method', title='Distribución de los métodos')

# Mostrar el gráfico
fig.show()

Peticiones % que llegan bien

In [12]:
peticiones = df[df['Method'] == 'GET']

In [13]:
peticiones.head()

,Client IP,Ident,Auth,Timestamp,Method,Request,HTTP Version,Status Code,Bytes Sent,Referrer,User Agent
0,"""192.168.4.164",-,-,22/Dec/2023:15:19:05 +0300,GET,/DVWA/,HTTP/1.1,200,2020,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
26,"""192.168.4.164",-,-,22/Dec/2023:15:19:07 +0300,GET,/DVWA/,HTTP/1.1,302,433,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
27,"""192.168.4.164",-,-,22/Dec/2023:15:19:07 +0300,GET,/DVWA/login.php,HTTP/1.1,200,1016,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
206,"""192.168.4.164",-,-,22/Dec/2023:15:19:08 +0300,GET,/DVWA/,HTTP/1.1,302,414,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
207,"""192.168.4.164",-,-,22/Dec/2023:15:19:08 +0300,GET,/DVWA/dvwa/,HTTP/1.1,200,675,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...


In [14]:
peticiones_status = peticiones['Status Code'].value_counts().reset_index()
peticiones_status

,Status Code,count
0,200,3808
1,404,770
2,500,163
3,302,55
4,301,18
5,400,11
6,403,1


In [27]:
total_peticiones = peticiones_status['count'].sum()

# Calcular el porcentaje de cada barra con dos decimales
peticiones_status['porcentaje'] = (peticiones_status['count'] / total_peticiones * 100).round(2)

# Crear el gráfico de barras
fig = px.bar(peticiones_status, x='Status Code', y='count', color='Status Code', title='Distribución de los Status Code de las peticiones',
             text=peticiones_status['porcentaje'].astype(str) + '%')  # Mostrar el porcentaje de cada barra con dos decimales como texto

# Mostrar el gráfico
fig.show()

Envios % que se hacen bien

In [28]:
envios = df[df['Method'] == 'POST']

In [29]:
envios.head()

,Client IP,Ident,Auth,Timestamp,Method,Request,HTTP Version,Status Code,Bytes Sent,Referrer,User Agent
938,"""192.168.4.164",-,-,22/Dec/2023:15:19:10 +0300,POST,/DVWA/login.php,HTTP/1.1,302,329,http://192.168.4.161/DVWA/,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
1843,"""192.168.4.164",-,-,22/Dec/2023:15:19:17 +0300,POST,/DVWA/setup.php,HTTP/1.1,302,335,http://192.168.4.161/DVWA/setup.php,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
2023,"""192.168.4.164",-,-,22/Dec/2023:15:19:19 +0300,POST,/DVWA/vulnerabilities/exec/,HTTP/1.1,200,1659,http://192.168.4.161/DVWA/vulnerabilities/exec/,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
2024,"""192.168.4.164",-,-,22/Dec/2023:15:19:19 +0300,POST,/DVWA/vulnerabilities/exec/,HTTP/1.1,200,1659,http://192.168.4.161/DVWA/vulnerabilities/exec/,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
2025,"""192.168.4.164",-,-,22/Dec/2023:15:19:19 +0300,POST,/DVWA/vulnerabilities/exec/,HTTP/1.1,200,1762,http://192.168.4.161/DVWA/vulnerabilities/exec/,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...


In [30]:
envios_status = envios['Status Code'].value_counts().reset_index()
envios_status

,Status Code,count
0,303,3407
1,200,1729
2,302,943
3,500,546
4,404,62
5,403,2


In [32]:
total_envios = envios_status['count'].sum()

# Calcular el porcentaje de cada barra con dos decimales
envios_status['porcentaje'] = (envios_status['count'] / total_envios * 100).round(2)

# Crear el gráfico de barras
fig = px.bar(envios_status, x='Status Code', y='count', color='Status Code', title='Distribución de los Status Code de los envios',
             text=envios_status['porcentaje'].astype(str) + '%')  # Mostrar el porcentaje de cada barra con dos decimales como texto

# Mostrar el gráfico
fig.show()

In [35]:
df

,Client IP,Ident,Auth,Timestamp,Method,Request,HTTP Version,Status Code,Bytes Sent,Referrer,User Agent
0,"""192.168.4.164",-,-,22/Dec/2023:15:19:05 +0300,GET,/DVWA/,HTTP/1.1,200,2020,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
1,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/,HTTP/1.1,200,263,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
2,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/.git/config,HTTP/1.1,404,139,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
3,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/.git/config,HTTP/1.1,404,139,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
4,"""192.168.4.164",-,-,22/Dec/2023:15:19:06 +0300,HEAD,/DVWA/.svn/all-wcprops,HTTP/1.1,404,139,-,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...
...,...,...,...,...,...,...,...,...,...,...,...
13617,"""192.168.4.25",-,-,22/Dec/2022:16:22:11 +0300,POST,/index.php/component/search/,HTTP/1.1,303,376,http://192.168.4.161/DVWA,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
13618,"""192.168.4.25",-,-,22/Dec/2022:16:33:44 +0300,POST,/index.php/component/search/,HTTP/1.1,303,417,http://192.168.4.161/DVWA,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
13619,"""192.168.4.25",-,-,22/Dec/2022:16:23:07 +0300,POST,/index.php/component/search/,HTTP/1.1,303,393,http://192.168.4.161/DVWA,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...
13620,"""192.168.4.25",-,-,22/Dec/2022:16:32:54 +0300,POST,/index.php/component/search/,HTTP/1.1,200,3054,http://192.168.4.161/DVWA,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...


In [34]:
# Ratio entre bytes enviados y recibidos
df['Ratio'] = df['Bytes Sent'] / df['Bytes Received']
# Visualizar el ratio en un gráfico
plt.figure(figsize=(10, 6))
plt.plot(df['Timestamp'], df['Ratio'])
plt.xlabel('Timestamp')
plt.ylabel('Ratio (Bytes Sent / Bytes Received)')
plt.title('Ratio de bytes enviados y recibidos a lo largo del tiempo')
plt.show()

KeyError: 'Bytes Received'

In [36]:
df['Method'].unique()

array(['GET', 'HEAD', 'NETSPARKER', 'POST', 'OPTIONS', 'PROPFIND'],
      dtype=object)

In [47]:
bytes_sent = 0
bytes_received = 0
for i in df:
    if (df['Method'] == 'GET') or (df['Method'] == 'HEAD') or (df['Method'] == 'NETSPARKER'):
        bytes_sent += df['Bytes Sent']
    else:
        bytes_received += df['Bytes Sent']


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [40]:
# Definir una función para calcular el ratio de bytes enviados y recibidos según el método HTTP
def calculate_ratio(group):
    method = group['Method'].iloc[0]
    if method == 'GET' or method == 'HEAD' or method == 'NETSPARKER':
        bytes_sent = group['Bytes Sent'].sum()
        bytes_received = group['Bytes Sent'].count()
    else:
        bytes_sent = group['Bytes Sent'].count()
        bytes_received = group['Bytes Sent'].sum()
    return bytes_sent / bytes_received

# Calcular el ratio de bytes enviados y recibidos para cada método HTTP
df_grouped = df.apply(calculate_ratio).reset_index(name='Ratio')

# Visualizar el ratio en un gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(df_grouped['Method'], df_grouped['Ratio'])
plt.xlabel('Método HTTP')
plt.ylabel('Ratio (Bytes Sent / Bytes Received)')
plt.title('Ratio de bytes enviados y recibidos por método HTTP')
plt.xticks(rotation=45)
plt.show()

KeyError: 'Method'